# Algorithmische Geometrie

In [1]:
import sys
import os
import string
import time
import re
import datetime
import codecs
import itertools
import numpy as np
import sympy as sp
import sympy.solvers as spsolvers
import pycurl
import matplotlib.pyplot as plt

from copy import deepcopy
from scipy.special import binom
from matplotlib import cm
from matplotlib.ticker import LinearLocator

# Ausgabe von Rechnername und Datum:
hostname = os.uname().nodename.split('.')[0].capitalize()
last_started = datetime.date.today()
print(f"Notebook wurde zuletzt gestartet {last_started} am Rechner {hostname}.")

Notebook wurde zuletzt gestartet 2025-03-10 am Rechner Fullmacstudio.


In [2]:
# OMIT Moodle-Output ausschalten

In [3]:
# Selbstgebastelte Module:
from mf.lecture import write_code_snippet, write_function_snippet, MF_ROOTPATH
from mf.texutils import make_standalone_pdf, find_occurrences
from mf.binary_trees import OTHER_SIDE, BinaryTreeNode, BinaryTree, get_list_of_nodes, draw_binary_tree, simple_key_in_circle, adhoc_for_kd_tree, AVLTreeNode, AVLTree, key_n_length_in_circle
from mf.geo import npp, get_line_equation, get_solution_xy, get_normalvector, \
get_angle, get_normalized, get_intersection, plane_line_bisector, PlaneRectangle
from mf.picture import PstricksPicture
from mf.decorator import ps_decorator_factory

# Root folder:
ROOTPATH = '/Users/mfulmek/ucloud/books/dmti/'
TESTPATH = '/Users/mfulmek/ucloud/books/test/'

# Decorator:
ps_dmti = ps_decorator_factory(rootpath=ROOTPATH)
# Für Testzwecke:
ps_test = ps_decorator_factory(rootpath=TESTPATH)

# Name und Nummer des Files für das aktuelle Kapitel:
NUMBER = 6
SECTION = 'structures'
SOURCE = os.path.join(ROOTPATH, "snippets", f'{SECTION}_snippets.ipynb')

# Keyword-Arguments für die Funktionsaufrufe:
COMMON = {
    'rootpath' : ROOTPATH,
    'the_source' : SOURCE,
    'only_python' : False
}

ONLY_PYTHON = {
    'rootpath' : MF_ROOTPATH,
    'the_source' : SOURCE,
    'only_python' : True
}

# Ausgabe von Kapitel:
print(f"Snippets für Kapitel 7: Algorithmische Geometrie.")

Snippets für Kapitel 7: Algorithmische Geometrie.


In [4]:
# MOODLE Moodle-Output einschalten

## Ebene Geometrie in Python, mit den Modulen ```numpy``` und ```sympy```.

Hier zum Beispiel ein paar Funktionen, die elementare geometrische Konzepte in der Ebene implementieren.

In [5]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!

# ACHTUNG: numpy arbeitet mit Winkeln in Radiant; Angaben in Grad
# muß man also umrechnen!
# Conversion of degree to radiant for numpy's trigonometric functions.
RAD2DEG = 180.0/np.pi
DEG2RAD = 1/RAD2DEG
# In our "floating-point-geometry", we cannot handle very small lengths:
EPSILON = 10 ** (-8)

# Points are numpy-arrays:
# numpy bietet sehr viel "Lineare-Algebra-Funktionalität" (Vektoren,
# Matrizen, Addition, Multiplikation, Norm, ...), die wir natürlich
# hier verwenden wollen.
# Der Stern vor dem Argument-Namen bedeutet: Diese Funktion übernimmt
# eine variable ANZAHL von Argumenten (in Form einer Liste). Der Witz
# der Sache ist: npp(x,y) und npp([x,y]) liefern beide dasselbe
# Ergebnis.
def npp(*args):
    "Convert arguments to np.array"
    if len(args) == 1:
        # Only one argument: Should be a list or tuple
        return np.array(list(args[0]), dtype=float)
    return np.array(list(args), dtype=float)

# Für Programme, die Punkte in graphische Darstellungen verwandeln,
# sind Zahlenausgaben in "scientific notation" (also 1.0E-3 statt 0.001)
# störend, daher vermeiden wir das:
def chop(num, epsilon = 10**(-5)):
    "Chop small numbers; to avoid scientific notation"
    # Nächste ganze Zahl an num:
    numi = np.round(num)
    if np.abs(numi-num) < epsilon:
        return f'{int(numi)}'
    # Formatierte Ausgabe mit 5 Nachkommastellen:
    return f'{num:0.5}'

# Ausgabe eines Punktes als String:
def out(p):
    "String representation of point p"
    return f'({chop(p[0])},{chop(p[1])})'

# (Euklidische) Norm (Länge) eines Vektors:
def get_norm(v):
    "Compute the norm of v"
    # Wir verwenden hier die bereits in numpy vorhandene Funktionalität
    # (get_norm ist also nur ein "wrapper" für np.linalg.norm)
    return np.linalg.norm(v)

# Bringe einen Vektor (ungleich Nullvektor) auf Länge 1:
def get_normalized(v_v):
    "return v_v/norm(v_v)"
    len_v = get_norm(v_v)
    # Vermeide Division durch sehr kleine Zahlen:
    if len_v > 10**(-8):
        return v_v/len_v
    # ELSE hier _implizit_, durch return in der if-clause:
    return None

# Normalvektor:
# For two-dimensional vectors (nicht Null), we can easily find a
# (normalized) normal vector:
def get_normalvector(v_v):
    "Rotate v_v by +pi/2"
    # numpy-Funktion flip dreht die Reihenfolge um, und die Multiplikation
    # v1*v2 erfolgt _komponentenweise_ (nicht verwechseln mit dem inneren
    # Produkt np.inner(vq, v2)!!"")
    return np.flip(v_v)*np.array((-1,1), dtype=float)

# Normalisierter Normalvektor
def get_normalized_normal(v_v):
    "Rotate v_v by +pi/2 and normalize"
    n_v = np.flip(v_v)*np.array((-1,1), dtype=float)
    return get_normalized(n_v)

# Eine Klasse, die Geraden in der Ebene implementiert: Übernimmt als
# "initialisierende Argumente" zwei Punkte (die nicht zu nahe sein
# sollten, um numerische Probleme zu vermeiden).
class PlaneLine:
    "Class: Line in the plane, determined by two (distinct) points."
    def __init__(self, p_p, p_q):
        if get_norm(p_p-p_q) < EPSILON:
            print('Points too close in PlaneLine.__init__()!!!')
        # Speichere "lokale Kopien" der übergebenen Punkte - würde
        # man hier einfach self.p_p = p_p schreiben, würde self.p_p
        # nur einen _Verweis_ auf den Punkt p_p enthalten (der später
        # geändert werden könnte)
        self.p_p = np.copy(p_p)
        self.p_q = np.copy(p_q)
        # Normalvektorgleichung der Geraden:
        self.equation = get_line_equation(p_p,p_q)
        # Normalisierter Richtungsvektor:
        self.v_v = get_normalized(p_q-p_p)
        # Normalisierter Normalvektor:
        self.n_v = get_normalized_normal(p_q-p_p)
        # Die rechte Seite der Normalvektorgleichung <x,n> = c
        self.c_c = np.inner(self.n_v, self.p_p)

    # String-representation einer Instanz dieser Klasse:
    def __str__(self):
        return f'{out(self.p_p)}-{out(self.p_q)}: {self.equation}.'

    # "Aufräum-Funktion", wenn ein Objekt dieser Klasse nicht
    # mehr gebraucht wird
    def __del__(self):
        pass

    # Auswertung der Gleichung in einem Punkt:
    def eval_equation(self, p_p):
        "Evaluate line equation for point p_p"
        return np.inner(self.n_v, p_p) - self.c_c

    # Fußpunkt des Lots:
    def basepoint(self,p_p):
        "Compute base point of orthogonal projection"
        normal_equation = get_line_equation(p_p,p_p+self.n_v)
        return get_solution_xy([self.equation, normal_equation])[0]

    # Spiegelung eines Punktes:
    def reflect(self,p_p):
        "Return reflection of point p_p at self"
        return 2*self.basepoint(p_p) - p_p

    # Parallele durch einen gegebenen Punkt
    def parallel(self,p_s):
        "Return parallel line through p_s"
        base_p = self.basepoint(p_s)
        return PlaneLine(
            self.p_p + (p_s - base_p), self.p_q + (p_s - base_p)
        )

    # Normale durch einen gegebenen Punkt
    def normal(self,p_p):
        "Return orthogonal line through p_p"
        return PlaneLine(p_p, p_p + self.n_v)

# Streckensymmetrale von zwei Punkten:
def plane_line_bisector(p,q):
    """Return line bisector (a.k.a. 'Streckensymmetrale') of
    points p, q."""
    return PlaneLine((p+q)/2, (p+q)/2 + get_normalized_normal(q-p))

In [6]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'geometry-preamble',
    **COMMON,
    the_caption = r'Hilfsfunktionen für Geometrie in der Ebene.',
    preamble = 'import numpy as np'
)

\snippet{geometry-preamble}
Process geometry-preamble


### Gleichungen und Gleichungssysteme lösen mit ```sympy```

In [7]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!
# Solve equations with sympy

# Bestimme die Gleichung der Geraden durch zwei Punkte in 
# Normalvektorform:
def get_line_equation(p_1, p_2):
    "Return the equation of the line through p_1 and p_2"
    # sympy-Symbole müssen als solche "extra definiert" werden
    x_1,x_2= sp.symbols('x_1,x_2')
    normal_vector = get_normalvector(p_2-p_1)
    # sympy bietet _symbolische_ Gleichungen (und Lösungsmethoden
    # für diese):
    return sp.Eq(
        x_1*normal_vector[0]
        +x_2*normal_vector[1]
        -np.inner(normal_vector, p_1)
        ,0
    )

# Gleichung eines Kreises:
def get_circle_equation(p_m, radius):
    """Return the equation of the circle with center p_m and
    radius"""
    x_1,x_2= sp.symbols('x_1,x_2')
    return sp.Eq((x_1-p_m[0])**2+(x_2-p_m[1])**2-radius**2,0)

# Löse System von Gleichungen (in REELLEN Zahlen):
def get_solution_xy(equations):
    """Solve the system of equations, return LIST of solutions
    (maybe empty or singleton)"""
    x_1,x_2= sp.symbols('x_1,x_2')
    solutions = spsolvers.solve(
        equations, (x_1,x_2),
        domain=sp.S.Reals
    )
    if isinstance(solutions,list):
        # Leere Liste oder Liste mit mindestens 2 Lösungen
        return [npp(p) for p in solutions]
    # Otherwise it should hold: type(solutions) == dict:
    # "Eine" Lösung, eventuell in Form einer Gleichung: dict
    if x_1 in solutions and x_2 in solutions:
        return [npp(solutions[x_1],solutions[x_2])]
    print("!!! get_solution_xy returns equation!!!")
    return sp.Eq(solutions[x_1],solutions[x_2])

def get_intersection(obj_a, obj_b):
    """Find points of intersection for two geometrical objects"""
    return get_solution_xy([obj_a.equation, obj_b.equation])

In [8]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'geometry-preamble2',
    **COMMON,
    the_caption = r'Lösen von Gleichungen mit sympy.',
    preamble = """import numpy as np
import sympy as sp
import sympy.solvers as spsolvers"""
)

\snippet{geometry-preamble2}
Process geometry-preamble2


## Konvexe Hülle einer Punktwolke (langsam und instabil).
 Wir bestimmen die konvexe Hülle einer (endlichen) Menge von Punkten
 als jenes konvexe \EM{Polygon}, in dessen abgeschlossenen Inneren alle
 Punkte liegen, durch einen einfachen geometrischen Algorithmus.


In [9]:
def convex_hull_slow(lop, epsilon=10 ** (-8)):
    """Bestimme die Eckpunkte des Polygons, das die konvexe Hülle der
    Punktliste lop darstellt (geordnet im mathematisch positiven
    Umlaufsinn). Um numerische Ungenauigkeiten abzufangen, setze epsilon
    auf einen kleinen Wert > 0."""
    #@ Wir bestimmen die konvexe Hülle einer (endlichen) Menge von
    #@ Punkten als jenes konvexe \EM{Polygon}, in dessen abgeschlossenen
    #@ Inneren alle Punkte liegen, durch einen einfachen geometrischen
    #@ Algorithmus.
    arrows = dict()
    for i,p_i in enumerate(lop):
        for j,p_j in enumerate(lop[i+1:], start=i+1):
            # Trägergerade der Punkte p_i, p_j
            t_g = PlaneLine(p_i,p_j)
            
            # Zähle die Punkte von lop, die "schwach links" bzw.
            # "schwach rechts" von der Trägergeraden liegen:
            count_left = count_right = 0
            for k,q in enumerate(lop):
                # Setze Punkt q in die Geradengleichung ein:
                eval_q = t_g.eval_equation(q)
                if eval_q > epsilon:
                    # Punkt liegt links:
                    count_left+=1
                elif eval_q < -epsilon:
                    # Punkt liegt rechts:
                    count_right+=1
                else:
                    # Punkt liegt auf der Geraden - aber liegt er auch
                    # zwischen p_i und p_j
                    proj = np.inner(t_g.v_v, q - p_i)
                    if -epsilon<=proj and proj<=get_norm(p_j-p_i)+epsilon:
                        count_left+=1
                        count_right+=1

            # Wenn alle Punkte auf einer Seite der Trägergeraden
            # oder auf der Verbindungsstrecke von p_i, p_j liegen,
            # dann ist die Kante p_i, p_j Teil des konvexen Hüll-Polygons:
            if count_right == len(lop):
                arrows[j] = i
            if count_left == len(lop):
                arrows[i] = j

    # Bringe die Eckpunkte in die richtige Reihenfolge:
    start = list(arrows.keys())[0]
    corners = [start]
    for i in range(len(arrows)-1):
        new = arrows[start]
        corners+= [new]
        start = new
    
    return corners

In [10]:
write_function_snippet(convex_hull_slow,
    **COMMON,
    the_caption=r'Konvexe Hülle einer Punktwolke (langsam und instabil).',
    preamble = """import numpy as np
import sympy as sp
import sympy.solvers as spsolvers"""
)

\snippet{convex-hull-slow}
Process convex-hull-slow


## Numerische Ungenauigkeit wirkt sich _wirklich_ aus!


In [11]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!

# Punkte als Liste von Paaren:
rawdata = [
    (0,0),(0,1),(0.5,2),(3,-0.2),(3.5,-0.4),
    (3,3),(-2,1.5),(-4,3.5),(-5,0.5),(-1,-2)
]
# Wir machen draus eine Liste von numpy-Punkten zum Testen:
testlist = [npp(p) for p in rawdata]
# In einem Jupyter-Notebook können wir mit der folgenden Zeile die Laufzeit
# empirisch ermitteln:
%timeit convex_hull_slow(testlist)
# Man erhält ein Ergebnis von folgender Form:
# 24.9 milliseconds +/- 131 microseconds per loop 
# (mean +/- std. dev. of 7 runs, 10 loops each)
# Die Berücksichtigung numerischer Ungenauigkeiten ist wirklich notwendig:
g = PlaneLine(testlist[4],testlist[5])
# Hier sollte "theoretisch" NULL herauskommen - tut es aber nicht, wegen 
# numerischer Ungenauigkeit!
g.eval_equation(testlist[5])
# Und tatsächlich funktioniert unser Algorithmus für die test-Liste nicht,
# wenn epsilon=0:
convex_hull_slow(testlist)
# liefert [4, 5, 7, 8, 9], aber
convex_hull_slow(testlist, epsilon=0)
# liefert [9, 4]

4.9 ms ± 32.5 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


[9, 4]

In [12]:
write_code_snippet(
    "\n".join((_ih[CELLNUMBER].split("\n"))[1:]), # Zuletzt durchgeführte Code-Zelle!
    'numeric-errors',
    **COMMON,
    the_caption = r'Numerische Ungenauigkeit wirkt sich \EM{wirklich} aus!.',
    preamble = """import numpy as np
import sympy as sp
import sympy.solvers as spsolvers"""
)

\snippet{numeric-errors}
Process numeric-errors


## Inkrementeller Algorithmus zur Bestimmung der konvexen Hülle.


In [13]:
# Hilfsfunktion: Bestimme, in welcher Halbebene Punkt X in bezug auf
# die Verbindungsgerade der Punkte P und Q liegt.
def side_of_line(Q, P, X):
    # Normalvektor von P-Q:
    NV = get_normalvector(P-Q)
    # Einsetzen in Geradengleichgung: Wenn das innere Produkt
    # < X-Q, NV > größer als Null ist, dann liegt X "links von
    # der Geraden (wenn man von P nach Q schaut) ...
    return np.inner(X - Q, NV)

def convex_hull(lop):
    """Bestimme die Eckpunkte des Polygons, das die konvexe Hülle der
    Punktliste lop darstellt (geordnet im mathematisch positiven
    Umlaufsinn): Berechne getrennt den oberen Teil und den unteren Teil
    der konvexen Hülle."""
    # Wandle die Liste in eine_lexikographisch geordnete_ 
    # (len(lop) x 2) - Matrix um, als erstes einfach: Mache eine
    # (len(lop) x 2) - Matrix aus der Liste:
    dummy = np.array(lop)
    # Als zweites: Transponiere diese Matrix
    dummy_T = dummy.T
    # Drittens: Bestimme jene _Permutation der Indices_, die die
    # Zeilen der transponierten Matrix in lexikographische Ordnung
    # bringt:
    indices = np.lexsort((dummy_T[1], dummy_T[0]))
    # Damit erhalten wir die Punkte (Zeilen der Matrix) in
    # lexikographischer Ordnung:
    lop_array = dummy[indices]
    
    # Bestimme obere/untere Hälfte der konvexen Hülle (also des
    # Polygons), von links nach rechts:
    upper_points = [lop_array[0], lop_array[1]]
    lower_points = [lop_array[0], lop_array[1]]
    # Längen dieser "Hälften"
    upl = 2
    lpl = 2
    # (D.h.: Die Vereinigungsmenge von upper_points und lower_points
    # entspricht der Menge der Eckpunkte des Polygons; der linkeste
    # und der rechteste Punkt in upper_points und lower_points stimmen
    # überein, also upper_points[0] == lower_points[0] und
    # upper_points[-1] == lower_points[-1].)

    # For-Schleife startet beim dritten Punkt in lop_array (Index 2) ...
    for p in lop_array[2:]:
        # Innerhalb der upper_points: Suche den _am weitesten links
        # liegenden_ Punkt q, sodass alle Punkte x _rechts_ von q
        # _unterhalb_ der Verbindungsgeraden p,q liegen. Dass dazu
        # die folgende einfache for-Schleife genügt, muß man sich
        # kurz geometrisch überlegen!
        for k,(q,x) in enumerate(
            zip(
                upper_points[-2::-1],
                upper_points[-1::-1]
            )
        ):
            if side_of_line(q, p, x) > 0:
                k = k-1
                break
                
        upper_points = upper_points[:upl-k-1]
        upper_points += [p]
        upl = len(upper_points)
    
        # Innerhalb der lower points: Suche den _am weitesten links
        # liegenden_  Punkt q, sodass alle Punkte x _rechts_ von q
        # _oberhalb_ der Verbindungsgeraden p,q liegen. Dass dazu die
        # folgende einfache for-Schleife genügt, muß man sich (wie
        # zuvor) kurz geometrisch überlegen!
        for k,(q,x) in enumerate(
            zip(
                lower_points[-2::-1],
                lower_points[-1::-1]
            )
        ):
            if side_of_line(q, p, x) < 0:
                k = k-1
                break
        
        lower_points = lower_points[:lpl-k-1]
        lower_points += [p]
        lpl = len(lower_points)

    return upper_points[:-1]+[ppp for ppp in lower_points[1:][::-1]]

In [14]:
write_function_snippet(side_of_line,
    **COMMON,
    the_caption=r"""In welcher Halbebene (in Bezug auf eine gegebene Gerade)
    liegt ein gegebener Punkt?""",
    preamble = 'import numpy as np'
)
write_function_snippet(convex_hull,
    **COMMON,
    the_caption=r'Inkrementeller Algorithmus zur Bestimmung der konvexen Hülle.',
    preamble = 'import numpy as np'
)

\snippet{side-of-line}
Process side-of-line
\snippet{convex-hull}
Process convex-hull


## Fortunes Algorithmus zur Bestimmung eines Voronoi-Diagramms

Zum Abschluss ein komplizierterer Algorithmus, für den wir einige der bisher bereitgestellten Funktionen brauchen.

In [15]:
# OMIT

Zum Testen: Eine "Punktwolke", implementiert als ```numpy``` array.

In [16]:
# Punktwolke zum Testen:
testpoints = np.array(
    [
        [0,0],
        [-2,-0.5],
        [0.2,-1],
        [1,1.5],
        [0.25,2],
        [0.25,-1.5],
        [1.5,0.75],
        [-1.75,1.5],
        [2,-1],
        [-1,0.5]
    ],
    dtype=float
)
testpoints

array([[ 0.  ,  0.  ],
       [-2.  , -0.5 ],
       [ 0.2 , -1.  ],
       [ 1.  ,  1.5 ],
       [ 0.25,  2.  ],
       [ 0.25, -1.5 ],
       [ 1.5 ,  0.75],
       [-1.75,  1.5 ],
       [ 2.  , -1.  ],
       [-1.  ,  0.5 ]])

Mit ```numpy``` Funktionalität können wir diese Liste von Punkten für unsere Zwecke geeignet sortieren:

In [17]:
# Punktwolke, lexikographisch sortiert "von oben nach unten, dann von rechts nach links":
cloud_of_points = testpoints[np.lexsort(testpoints.T)[::-1]]
cloud_of_points

array([[ 0.25,  2.  ],
       [ 1.  ,  1.5 ],
       [-1.75,  1.5 ],
       [ 1.5 ,  0.75],
       [-1.  ,  0.5 ],
       [ 0.  ,  0.  ],
       [-2.  , -0.5 ],
       [ 2.  , -1.  ],
       [ 0.2 , -1.  ],
       [ 0.25, -1.5 ]])

In [18]:
np.append(cloud_of_points, np.array([[0,-10**6]], dtype=float), axis=0)

array([[ 2.50e-01,  2.00e+00],
       [ 1.00e+00,  1.50e+00],
       [-1.75e+00,  1.50e+00],
       [ 1.50e+00,  7.50e-01],
       [-1.00e+00,  5.00e-01],
       [ 0.00e+00,  0.00e+00],
       [-2.00e+00, -5.00e-01],
       [ 2.00e+00, -1.00e+00],
       [ 2.00e-01, -1.00e+00],
       [ 2.50e-01, -1.50e+00],
       [ 0.00e+00, -1.00e+06]])

In [19]:
# MOODLE

### Wir verwenden doppelt verkettete Listen

Wiederholung aus dem vorigen Kapitel:

In [20]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!
# Ein "Hilfs-Dictionary" für eine "Spiegelung"
# (Vertauschung von links und rechts), die sich hier
# als nützlich erweist:
mirror = {'left' : 'right', 'right' : 'left'}

class DLLNode:
    """A node to be used in a doubly linked list"""
    def __init__(self, data):
        # Store the data
        self.contents = data
        # Initially, the "pointers" to the left/right neighbours
        # are None
        self.neighbour = {'left' : None, 'right' : None}

    def __str__(self):
        """String representation of the node"""
        return f'{self.contents} {self.neighbour}'

    # "Hineinquetschen" eines neuen Knotens, links oder rechts
    # von self:
    def squeeze_in(self, what, where):
        """Insert a new node (containing "what") to side "where".
        """
        # Make the new node containing "what":
        new_node =  DLLNode(what)
        # Self's neighbour on side "where" becomes new node`s neighbour:
        new_node.neighbour[where] = self.neighbour[where]
        if self.neighbour[where]:
            self.neighbour[where].neighbour[mirror[where]] = new_node
        # Make this new node self's neighbour on side "where":
        self.neighbour[where] = new_node
        new_node.neighbour[mirror[where]] = self
        return new_node

    # Entfernen von self aus seiner "Nachbarschaft":
    def detach(self):
        """Remove self."""
        # Nachbarn von self:
        left, right = [self.neighbour[where] for where in ["left", "right"]]
        if left:
            left.neighbour["right"] = right
        if right:
            right.neighbour["left"] = left
        return self

In [21]:
CELLNUMBER = len(_ih) - 1 # Trick, um die Nummer dieser Zelle zu speichern!
class DoublyLinkedList:
    """Implementation of a doubly linked list."""
    def __init__(self):
        # Initially, the list is empty: Pointers to
        # left and right ends are None.
        self.end = {'left' : None, 'right' : None}
        self.length = 0

    def apply(self, the_func, *additional_arguments):
        """Traverse the list from left to right and
        apply function the_func to the data stored;
        return list of results."""
        # Beachte die Möglichkeit, der Funktion the_func
        # eine _variable_ Zahl zusätzlicher Argumente
        # zu übergeben (0 ist auch möglich!)
        results = []
        pointer = self.end['left']
        while pointer:
            results+= [the_func(pointer, *additional_arguments)]
            pointer = pointer.neighbour['right']
        return results

    # Beispiel, wie "apply" verwendet werden kann:
    def list_of_nodes(self):
        """Return the doubly linked list as a "normal" python
        list of nodes"""
        return self.apply(lambda x : x)

    # Noch ein Beispiel:
    def __str__(self):
        """String representation of the list"""
        data_list = self.apply(lambda x : f'{x.contents}')
        return f"DLL of length ({self.length}): (" + ", ".join(data_list)+")"

    # Anfügen bzw. Entfernen von Listen heißt "traditionell"
    # "push" bzw. "pop": Bei einer doppelt verketteten Liste
    # können wir links oder rechts anfügen bzw. entfernen.
    def push(self, data, where='right'):
        """Push (i.e., append) new data at list's end "where"."""
        if self.end[where]:
            new_end = self.end[where].squeeze_in(data, where)
        else:
            new_end = DLLNode(data)
            self.end[mirror[where]] = new_end

        self.length+= 1
        self.end[where] = new_end

    def pop(self, where='right'):
        """Remove node and return its data at list's end "where"."""
        if self.end[where]:
            # Funktion "detach" gibt den entfernten Knoten zurück:
            return self.end[where].detach().contents
            self.length-= 1
        # Implicit else:
        print('list is empty!!!')
        return None

### Schnittpunkte zweier Parabel mit Brennpunkten ```p1```, ```p2``` und gemeinsamer Leitlinie $y=$```h```.


In [22]:
def parabolic_intersections(p1, p2, h):
    """Return the x-coordinates of the (two, in general) intersections
    of parabolas with focus/directrix (p1,y=h) and (p2,y=h):"""
    a,b = p1
    c,d = p2
    if b < h or d < h:
        print(f'Invalid input: {p1}, {p2} are not both above line y = {h}.')
        return None
    
    if d == h:
        # In this case, the parabola is degenerate: A vertical ray starting
        # upwards in (c,d).
        return [c]*2

    if b == h:
        # In this case, the parabola is degenerate: A vertical ray starting
        # upwards in (a,b).
        return [a]*2

    if b == d:
        # In this case, there is only _one_ point of intersection:
        return [(a+c)/2]*2
    
    # Implicit else: We expect _two_ points of intersection!
    discriminant = ((a-c)**2 + (b-d)**2)/((b-h)*(d-h))
    sqareroot_with_factor = (d-h)*np.sqrt(discriminant)
    c1, c2 = c - sqareroot_with_factor, c + sqareroot_with_factor
    
    # Return the x-coordinates of the (two) points of intersection:
    return sorted([((a*(h-d) + (b-h)*cc)/(b-d)) for cc in [c2, c1]])

In [23]:
write_function_snippet(parabolic_intersections,
    **COMMON,
    the_caption=r'Schnittpunkte zweier Parabel mit Brennpunkten \pythoncode{p1}, \pythoncode{p2} und gemeinsamer Leitlinie $y=$\pythoncode{h}.',
    preamble = ''
)

\snippet{parabolic-intersections}
Process parabolic-intersections


### Hilfsfunktion: Bilden drei aufeinanderfolgende Parabel-Bögen einer Strandlinie ein circle event?

Falls ja, gib _Höhe_ des events und _Mittelpunkt_ des entsprechenden Kreises (als ```tuple```: Dient auch als ```key``` im Dictionary der circle events!) zurück.


In [24]:
def gives_circle_event(A, X, B):
    """Consider the three foci A, X, B of three consecutive arcs of
    a beachline: Is there a circle event affecting the "central" arc
    (with focus X)? If so, return the vertex and key corresponding
    to this event."""
    if np.inner(get_normalized_normal(X-A),B-X) >= 0:
        return None, None
    # Implicit else:
    bisector_ax = plane_line_bisector(A,X)
    bisector_bx = plane_line_bisector(X,B)
    vertex = get_intersection(bisector_ax, bisector_bx)[0]
    radius = get_norm(X-vertex)
    x,y = vertex
    key = y-radius
    # Return key (height where this event should be handled)
    # and center (vertex) _as a tuple_:
    return key, (x,y)

In [25]:
write_function_snippet(gives_circle_event,
    **COMMON,
    the_caption=r'Hilfsfunktion: Bilden drei aufeinanderfolgende Parabel--Bögen einer Strandlinie ein circle event? Falls ja, gib Höhe des events und Mittelpunkt des entsprechenden Kreises (als \pythoncode{tuple}).',
    preamble = ''
)

\snippet{gives-circle-event}
Process gives-circle-event


## Konstruiere das Voronoi-Diagramm einer Punktwolke

Wir sortieren die Punktwolke mit einer ```numpy```-Funktion lexikographisch "von oben nach unten, dann von rechts nach links": Dann entsprechen die site-events, die beim "Herabsinken der Leitlinie" entstehen, einfach den Punkten der Wolke in dieser Reihenfolge.

Bevor wir aber ein solches site-event einfügen, müssen wir alle "höher liegenden" circle-events berücksichtigen, die die Strandlinie verändern.

In [45]:
def make_voronoi_diagram(cloud, verbose=False):
    """Compute the Voronoi-diagram for the cloud of points,
    assumedly given as n x 2 array. We assume that the points
    are _pairwise disjoint_!"""
    # Als erstes: Punktwolke, lexikographisch sortiert
    # "von oben nach unten, dann von rechts nach links":
    cop = cloud[np.lexsort(cloud.T)[::-1]]

    # Aus Gründen, die später klar werden, geben wir noch
    # einen "ganz weit unten" liegenden Punkt dazu:
    MINUSINFINITY = -10**9
    cop = np.append(cop, np.array([[0,MINUSINFINITY]], dtype=float), axis=0)

    # Die beachline ist gegeben durch Liste der INDICES der Brennpunkte
    # (können sich wiederholen!), zu Beginn besteht sie also nur
    # aus den ersten Brennpunkten mit Index 0 und 1:
    beachline = DoublyLinkedList()
    if cop[0][1] == cop[1][1]:
        # Die ersten Brennpunkte liegen auf derselben Höhe,
        # der zweite (Index 1) liegt links vom ersten (Index 0):
        beachline.push(1)
        beachline.push(0) # ... "von rechts nach links sortiert!"
    else:
        # Der zweite Brennpunkt liegt unterhalb, die durch ihn
        # bestimmte Parabel "durchstößt von unten" die durch den
        # ersten Brennpunkt bestimmte Parabel:
        beachline.push(0)
        beachline.push(1)
        beachline.push(0)

    def str_beachline():
        return "("+",".join(beachline.apply(lambda x : str(x.contents)))+")"
        
    # Wir bauen das Voronoi-Diagramm auf als ein Dictionary,
    # dessen keys (geordnete) Paare von Indices von Punkten
    # sind, deren Streckensymmetralen Kanten des Diagramms
    # enthalten, und dessen values die Punkte (oder: der Punkt)
    # sind, die diese Kanten begrenzen.
    voronoi_diagram = dict()
    # Hilfsfunktion: Sorge dafür, dass ein Tupel (i,j)
    # geordnet wird (also i<j gilt).
    
    def edge_key(i, j):
        """Key for edge bordering two cells with centers
        cloud[i], cloud[j] in the Voronoi diagram (i.e.:
        edge is part of the bisector of line (cloud[i], cloud[j]).
        """
        return (i,j) if i<j else (j,i)
        
    # Arbeite nun der Reihe nach die "site-events" ab, die durch
    # die restlichen Punkte der cloud of points (cop) gegeben sind:
    # Durch die lexikographische Sortierung entspricht das einem
    # "Herabsinken der Leitlinie von oben nach unten":
    old_site_event_height = cop[1][1]
    if verbose:
        print(f"Initial site height: {old_site_event_height}")
    for point_index,point in enumerate(cop[2:], start=2):
        site_event_height = point[1]
        if verbose:
            print(f"\tNew site-event {point_index}: {point}?")
            print(str_beachline())
        # Als erstes müssen wir aber schauen, ob es circle events
        # _oberhalb_ dieser site_event_height_ gibt; dazu muss
        # die beachline zumindest Länge 3 haben:
        if beachline.length > 2:
            left = beachline.end["left"]
            middle = left.neighbour["right"]
            right = middle.neighbour["right"]
            while right:
                # Indices der Brennpunkte:
                li, mi, ri = [
                    node.contents for node in [left, middle, right]
                ]
                # Einen eindeutigen Schnittpunkt der Streckensymmetralen
                # von (li, mi) und (mi, ri) gibt es natürlich nur, wenn
                # li != ri ist.
                if li != ri:
                    # Berechne den Schnittpunkt der Streckensymmetralen und
                    # die Höhe, auf der das entsprechende circle-event
                    # passiert:
                    circle_event_height, center = gives_circle_event(
                        cop[li], cop[mi], cop[ri]
                    )
                    
                    if (
                        (not circle_event_height is None) 
                        and 
                        (old_site_event_height >
                         circle_event_height >=
                         site_event_height
                        )
                    ):
                        # Dieses circle-event müssen wir nun berücksicht-
                        # igen: Kanten des Voronoi-Diagramms "codieren"
                        # wir als das (geordnete) Paar der Indices der
                        # Punkte, deren  Streckensymmetrale die Kante
                        # enthält.
                        for key in [
                            edge_key(li, mi),
                            edge_key(mi, ri),
                            edge_key(li, ri)
                        ]:
                            if key not in voronoi_diagram:
                                # Diese Kante hat (vorläufig) nur einen
                                # Endpunkt:
                                voronoi_diagram[key] = [center]
                            else:
                                # Diese Kante hat einen zweiten Endpunkt:
                                voronoi_diagram[key].append(center)
                        # Der "mittlere" Punkt wird nun aus der beachline
                        # entfernt:
                        middle.detach()
                        # Nächsten Durchlauf der while-Schleife vorbereiten
                        # (Knoten "middle" wurde ja grade entfernt):
                        middle = left
                        if verbose:
                            print(f"circle_event h = {li,mi,ri} : {center}")
                            print(str_beachline())
                # Nächsten Durchlauf der while-Schleife vorbereiten:
                left = middle
                middle = right
                right = middle.neighbour["right"]
                
        # Ok: Alle circle-events, die oberhalb des aktuell betrachteten
        # site-events auftreten, wurden behandelt: Wir suchen jetzt die
        # Stelle, wo der neue Brennpunkt in die beachline eingefügt werden
        # soll - aber NUR, wenn wir nicht unseren ganz oben eingefügten
        # "zusätzlichen" Punkt erreicht haben, denn dann sind wir fertig
        # (denn alle circle-events haben wir ja grade erledigt):
        if site_event_height <= MINUSINFINITY:
            return voronoi_diagram
        # Implicit else: Wir haben noch einen "regulären" Punkt, dessen
        # site-event wir nun abarbeiten müssen.
        x_coordinate = point[0]
        left_node = beachline.end["left"]
        site_event_done = False
        while (right_node := left_node.neighbour["right"]):
            # Die Indices der Brennpunkte ...
            left_focus_index = left_node.contents
            right_focus_index = right_node.contents
            # ... und die Brennpunkte selbst:
            left_focus = cop[left_focus_index]
            right_focus = cop[right_focus_index]
            # The arc with the _lower_ focus "cuts from below";
            # this determines which of the two points of intersections
            # we should choose:
            left_or_right = 0 if left_focus[1] >= right_focus[1] else 1
            # Die x-Koordinate des Schnittpunkts
            intersection_x = parabolic_intersections(
                left_focus,
                right_focus,
                site_event_height
            )[left_or_right]
            if x_coordinate > intersection_x:
                left_node = right_node
                continue
            elif x_coordinate == intersection_x:
                # Der neu einzufügende Brennpunkt liegt "genau zwischen"
                # dem linken und dem rechten Parabelbogen:
                left_node.squeeze_in(point_index, "right")
                site_event_done = True
                break
            else: # D.h, x_coordinate < intersection_x:
                # Der neu einzufügende Brennpunkt "zerschneidet" den
                # linken Parabelbogen, der nun also _zwei_ Parabelbögen
                # zur beachline beisteuert:
                left_node.squeeze_in(point_index, "right")
                right_node.squeeze_in(left_focus_index, "left")
                site_event_done = True
                break
            # Nicht vergessen: Nachsten Durchlauf der while-Schleife
            # vorbereiten!
            left_node = right_node

        if not site_event_done:
            # Der neu einzufügende Brennpunkt "zerschneidet"
            # den letzten Parabelbogen:
            last_focus_index = beachline.end["right"].contents
            beachline.push(point_index)
            beachline.push(last_focus_index)
            
            
        if verbose:
            print(str_beachline())

        old_site_event_height = site_event_height
    
    return voronoi_diagram

In [46]:
CAPTION = """Funktion zur Konstruktion des Voronoi--Diagramms
für eine Punktwolke: Die Punkte werden zuerst lexikographisch
\\anf{von oben nach unten, dann von rechts nach links} sortiert.
Durch Abarbeiten dieser Punkte in einer \\pythoncode{for}--Schleife
wird also automatisch das \\anf{Herabsinken der Leitlinie} für die
site--events richtig umgesetzt; aber natürlich müssen zwischendurch
allfällige circle--events behandelt werden. Das Voronoi--Diagramm
selbst wird als \\pythoncode{dict} konstruiert, dessen \\pythoncode{keys}
die Paare von (Indices von) Punkten sind, deren Streckensymmetrale eine
Kante $e$ des Voronoi--Diagramms enthält, und dessen \\pythoncode{values}
Punkte (also Knoten) des Voronoi--Diagramms sind, die diese Kante $e$ begrenzen
(zumindest \\anf{auf einer Seite}). In den Kommentarzeilen
ist die Vorgangsweise genau beschrieben (darum ist der Code auch so lang).
"""
write_function_snippet(make_voronoi_diagram,
    **COMMON,
    the_caption= CAPTION.replace("\n", " "),
    preamble = ''
    # make_pdf=False
)

\snippet{make-voronoi-diagram}
Process make-voronoi-diagram


Das Voronoi-Diagramm liefert auch den Rand der konvexen Hülle (und diesen Rand zu kennen ist nützlich, wenn man das Diagramm zeichnen will):

In [28]:
def convex_hull_voronoi(voronoi):
    # Die keys (Paare von Indices von Punkten) im so berechneten
    # Voronoi-Diagramm, die _nur einen_ zugeordneten Endpunkt haben,
    # bestimmen die Punkte, die den Rand der konvexen Hülle der
    # Punktwolke bilden:
    convex_hull_edges = [
        set(key) for key, value in voronoi.items() if len(value) < 2
    ]
    # Diese Liste von zweielementigen Teilmengen von Indices können
    # wir als Kanten in einem Graphen interpretieren, dessen Knoten
    # die _Indices_ der Punkte am Rand der Punktwolke sind, für die
    # das Voronoi-Diagramm berechnet wurde.
    # Diese Knoten wollen wir aber noch in die richtige Reihenfolge
    # bringen, sodass die so geordneten Knoten einem Kreis im Graphen
    # entsprechen):
    current_edge = convex_hull_edges[0]
    # Ok: Die Knoten dieser Kante liegen auf dem gesuchten Kreis:
    convex_hull_points = list(current_edge)
    # Diese Kante entfernen wir aus der Liste:
    convex_hull_edges.remove(current_edge)
    # Jetzt vervollständigen wir den Kreis:
    v0,v1 = current_edge
    while convex_hull_edges:
        for edge in convex_hull_edges:
            if v1 in edge:
                # "edge" enthält Knoten v1 ...
                convex_hull_edges.remove(edge)
                # ... und mit dem _anderen_ Knoten setzen wir fort:
                v1 = list(edge.difference(set([v1])))[0]
                convex_hull_points.append(v1)
    return convex_hull_points

In [30]:
CAPTION = """Das Voronoi--Diagramm für eine Punktwolke liefert auch den \\anf{Rand}
der konvexen Hülle: Dazu sammeln wir einfach die Kanten des Voronoi--Diagramms,
die \\EM{Strahlen} (also \\EM{nicht Strecken}!) entsprechen und ordnen Punkte der Wolke, die
diesen Kanten entsprechen, in einem Kreis (im graphentheoretischen Sinn) an.
Die hier gezeigte Implementierung dieser Idee passt zu der Konstruktion des
Voronoi--Diagramms als \\pythoncode{dict}  in \\pythoncode{make\\_voronoi\\_diagram}.
"""
write_function_snippet(convex_hull_voronoi,
    **COMMON,
    the_caption= CAPTION.replace("\n", " "),
    preamble = ''
    # make_pdf=False
)

\snippet{convex-hull-voronoi}
Process convex-hull-voronoi


In [ ]:
# OMIT

In [48]:
voronoi = make_voronoi_diagram(testpoints)

In [49]:
@ps_dmti
def show_voronoi(cloud, voronoi, style=None):
    """Zeichne das Voronoi--Diagramm:?"""
    # Als erstes: Punktwolke, lexikographisch sortiert
    # "von oben nach unten, dann von rechts nach links":
    cop = cloud[np.lexsort(cloud.T)[::-1]]

    # Ausdehnung der Punktwolke:
    x_min = np.floor(cop.T[0].min()-0.5)
    x_max = np.ceil(cop.T[0].max()+0.5)
    y_min = np.floor(cop.T[1].min()-0.5)
    y_max = np.ceil(cop.T[1].max()+0.5)

    # left lower and right upper corner of rectangle
    LL = np.floor(npp(x_min,y_min))
    RU = np.ceil(npp(x_max, y_max))
    rectangle = PlaneRectangle(LL,RU)

    ps = PstricksPicture(unit=1.25)

    # Grid, zur Orientierung
    ps.grid_axes_ticks(np.floor(npp(x_min,y_min)), np.ceil(npp(x_max, y_max)))
    
    # Indices der Punkte am Rand der konvexen Hülle:
    cvi = convex_hull_voronoi(voronoi)
    # Wir suchen einen Punkt _innerhalb_ der konvexen Hülle:
    inner_point_index = list(set(range(len(cop))).difference(set(cvi)))[0]
    inner_point = cop[inner_point_index]

    # Liegt ein Voronoi-Punkt, der einen _Strahl_ im Voronoi-Diagramm
    # begrenzt, innerhalb oder ausserhalb der konvexen Hülle?
    def inside_convex_hull(i,j,P):
        # Normalvektor der Verbindungsgeraden:
        nv = get_normalized_normal(cop[i] - cop[j])
        # Rechte Seite der Normalvektorgleichung:
        c = np.inner(cop[i], nv)
        return (np.inner(nv,P)-c)*(np.inner(nv,inner_point)-c) >= 0
    
    # Zeichne die konvexe Hülle:
    ps.polygon(
        [cop[i] for i in cvi[:-1]],
        options="fillstyle=solid,fillcolor=honeydew,opacity=0.4,linecolor=lightgray")

    # Die Punkte der Punktwolke:
    for i,P in enumerate(cop):
        ps.geopoint(P)
        ps.rput(P+npp(0.175,0.175), f"{{\\tiny ${i}$}}")

    # Jetzt das Voronoi-Diagramm:
    ps.auxlinestyle()
    for key, value in voronoi.items():

        # Kanten zeichnen:
        if len(value) > 1:
           ps.line([npp(P) for P in value], options="linecolor=blue")
        else:
            i,j = key
            P = npp(value[0])
            M = (cop[i]+cop[j])/2
            rv = get_normalized(M-P)
            factor = 1 if inside_convex_hull(i,j,P) else -1
            # Bestimme Schnittpunkt des Strahls, der der Kante (i,j)
            # entspricht, mit dem umgebenden Rechteck:
            if (Q := rectangle.intersect_with_beam(P,P+factor*rv)) is not None:
                ps.arrow(P,Q, options="linecolor=blue")
    
        # Punkte zeichnen:
        for P in value:
            ps.geopoint(npp(P), color="blue")
            
    ps.extend(0.5*npp(1,1), 0.1*npp(1,1))
    return ps

make_standalone_pdf(*show_voronoi(testpoints, voronoi), showthis=True)

% show_voronoi(2025-03-10): (-3.5,-2.5) x (3.1,3.1)
\cps{show_voronoi}
Size of picture (width,height) in cm: (8.25, 7.00)
Process: /Users/mfulmek/ucloud/books/dmti/graphics/show_voronoi.tex
Changed path from /Users/mfulmek/ucloud - Markus Fulmek (fulmekm4)@ucloud.univie.ac.at/books/dmti/tmp to /Users/mfulmek/ucloud/books/dmti/tmp.
Run command "latex --shell-escape standalone.tex".
Run command "dvipdf -dALLOWPSTRANSPARENCY -dAutoRotatePages=/None standalone.dvi".
Remove standalone.aux".
Remove standalone.log".
Remove standalone.dvi".
Try to show standalone.pdf:
document 11
Return to old path /Users/mfulmek/ucloud - Markus Fulmek (fulmekm4)@ucloud.univie.ac.at/books/dmti/tmp.
Move standalone.pdf to /Users/mfulmek/ucloud/books/dmti/graphics_pdf/show_voronoi.pdf
Remove standalone.tex.
Remove _minted-standalone.
